In [ ]:
### imports

# external modules
import sys
import os
import numpy as np
from numpy.random import default_rng
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [ ]:
### load the data

name = 'data_s32x32_n10k'
dataname = name+'_images.npy'
with open(dataname,'rb') as f:
    ims = np.load( f )
labelname = name+'_labels.npy'
with open(labelname,'rb') as f:
    ntrue = np.load( f )
print('loaded data "{}" of shape {}'.format(dataname,ims.shape))
print('loaded labels "{}" of shape {}'.format(labelname,ntrue.shape))

In [ ]:
### process data

image_size = ims[0].shape
nimages = len(ims)
ntrain = int(nimages*0.8)
ims_train = ims[:ntrain]
ims_train = ims_train.reshape(-1, image_size[0], image_size[1], 1)
ntrue_train = ntrue[:ntrain]
ims_test = ims[ntrain:]
ims_test = ims_test.reshape(-1, image_size[0], image_size[1], 1)
ntrue_test = ntrue[ntrain:]
print('image size: {}'.format(image_size))
print('number of training images: {}'.format(len(ims_train)))
print('number of testing images: {}'.format(len(ims_test)))

In [ ]:
### define the convolutional network

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1))

model.summary()

model.compile(optimizer='adam', loss='mse')

In [ ]:
### define plotting functions

def plot_model_prediction(datalist, labellist=None, colorlist=None,
                          xaxtitle='Predicted minus true count', xaxtitlesize=None, 
                          yaxtitle='Number of instances (normalized)', yaxtitlesize=None):
    
    fig,axs = plt.subplots(nrows=1, ncols=2, squeeze=False, figsize=(12,6))
    
    # make plot of absolute difference
    halfwindow = 5
    bins = np.arange(-halfwindow-0.5,halfwindow+0.5,1)
    ax = axs[0,0]
    for i,(npred,ntrue) in enumerate(datalist):
        diff = npred-ntrue
        ax.hist(diff, bins=bins, histtype='step', linewidth=3,
                label=labellist[i], color=colorlist[i], 
                density=True)
    ax.set_xticks(np.arange(-halfwindow,halfwindow,1))
    ax.legend()
    if xaxtitle is not None: ax.set_xlabel(xaxtitle, fontsize=xaxtitlesize)
    if yaxtitle is not None: ax.set_ylabel(yaxtitle, fontsize=yaxtitlesize)
    
    # make plot of relative difference
    relbins = np.linspace(0, 2, num=20)
    ax = axs[0,1]
    for i,(npred,ntrue) in enumerate(datalist):
        reldiff = np.where(ntrue<1, 0, np.abs(np.divide(npred-ntrue,ntrue)))
        ax.hist(reldiff, bins=relbins, histtype='step', linewidth=3,
                label=labellist[i], color=colorlist[i], 
                density=True)
    ax.legend()
    if xaxtitle is not None: ax.set_xlabel(xaxtitle+' (relative)', fontsize=xaxtitlesize)
    if yaxtitle is not None: ax.set_ylabel(yaxtitle, fontsize=yaxtitlesize)
    return (fig,ax)

def plot_loss(data, xlims=None,
              title=None, titlesize=None, 
              xaxtitle='Epoch', xaxtitlesize=None, 
              yaxtitle='Loss', yaxtitlesize=None,
              legendsize=None, legendloc='best',
              doshow=True):
    ### plot the training and validation loss of a keras/tensorflow model
    # input arguments:
    # - data: the object returned by the .fit method when called upon a keras model
    # - other: plot layout options
    fig,ax = plt.subplots()
    if 'loss' in data.history.keys(): ax.plot(data.history['loss'], linestyle=(0,()), color="#1A237E", linewidth=3, label='Training')
    if 'val_loss' in data.history.keys(): ax.plot(data.history['val_loss'], linestyle=(0,(3,2)), color="#4DB6AC", linewidth=3, label='Validation')
    ax.legend(loc=legendloc, fontsize=legendsize)
    ax.set_yscale('log')
    if xlims is not None: ax.set_xlim(xlims)
    if title is not None: ax.set_title(title, fontsize=legendsize)
    if xaxtitle is not None: ax.set_xlabel(xaxtitle, fontsize=xaxtitlesize)
    if yaxtitle is not None: ax.set_ylabel(yaxtitle, fontsize=yaxtitlesize)
    if doshow: plt.show(block=False)
    return (fig,ax)

In [ ]:
### make a plot of the model prediction before training

npred_train = np.squeeze(model.predict( ims_train ))
npred_test = np.squeeze(model.predict( ims_test ))

fig,axs = plot_model_prediction( datalist=[(npred_train,ntrue_train),(npred_test,ntrue_test)],
                                 labellist = ['Training','Testing'],
                                 colorlist = ['b','m'] )

In [ ]:
### do the training

history = model.fit(ims_train, ntrue_train, epochs=10, validation_split=0.2)
fig,ax = plot_loss(history)

In [ ]:
### make a plot of the model prediction after training

npred_train = np.squeeze(model.predict( ims_train ))
npred_test = np.squeeze(model.predict( ims_test ))

fig,axs = plot_model_prediction( datalist=[(npred_train,ntrue_train),(npred_test,ntrue_test)],
                                 labellist = ['Training','Testing'],
                                 colorlist = ['b','m'] )

In [ ]:
### plot some examples

nplot = 5
rng = default_rng()
inds = rng.integers(low=0, high=len(ims_test), size=nplot)
for indx in inds:
    fig,ax = plt.subplots( figsize=(6,6) )
    ax.imshow( ims_test[indx], cmap='gray' )
    plt.show()
    print('Precicted count: {}, true count: {}'.format(int(round(npred_test[indx])),int(ntrue_test[indx])))